In [1]:
import tensorflow as tf

In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


## Layers: common sets of useful operations

Tensorflow provides both a set of many common layers as a well as easy ways for us to write **our own application-specific layers** either **from scratch** or as the composition of existing layers.

In [3]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [5]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
layer.variables

[<tf.Variable 'dense/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.46782422, -0.05257368, -0.25427076,  0.32925683, -0.09659088,
          0.32908106, -0.13641864,  0.2193982 ,  0.4319312 , -0.33939832],
        [ 0.04265887,  0.20727038, -0.24140438,  0.05396444,  0.23715138,
          0.5543944 , -0.13134867,  0.3376338 , -0.563634  ,  0.27744722],
        [-0.20655927, -0.171323  ,  0.23290563, -0.10670441,  0.55393   ,
         -0.4798702 ,  0.38878685,  0.00894934,  0.23586816,  0.2853368 ],
        [ 0.3723325 ,  0.24434882, -0.51813656,  0.5341199 ,  0.5725302 ,
          0.09141684,  0.58302504,  0.4946534 , -0.44481516,  0.39735776],
        [-0.1548793 , -0.13500199, -0.54266936, -0.31376085,  0.5232113 ,
         -0.04429144,  0.5632487 ,  0.59331197, -0.536319  ,  0.34284735]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
layer.kernel

<tf.Variable 'dense/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.46782422, -0.05257368, -0.25427076,  0.32925683, -0.09659088,
         0.32908106, -0.13641864,  0.2193982 ,  0.4319312 , -0.33939832],
       [ 0.04265887,  0.20727038, -0.24140438,  0.05396444,  0.23715138,
         0.5543944 , -0.13134867,  0.3376338 , -0.563634  ,  0.27744722],
       [-0.20655927, -0.171323  ,  0.23290563, -0.10670441,  0.55393   ,
        -0.4798702 ,  0.38878685,  0.00894934,  0.23586816,  0.2853368 ],
       [ 0.3723325 ,  0.24434882, -0.51813656,  0.5341199 ,  0.5725302 ,
         0.09141684,  0.58302504,  0.4946534 , -0.44481516,  0.39735776],
       [-0.1548793 , -0.13500199, -0.54266936, -0.31376085,  0.5232113 ,
        -0.04429144,  0.5632487 ,  0.59331197, -0.536319  ,  0.34284735]],
      dtype=float32)>

In [8]:
layer.bias

<tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

## Implementing custom layers

The best way is extending the `tf.keras.Layer` class and implementing:
- `__init__`, where we can do all **input-independent** initialization
- `build`, where we know the shapes of the input tensors and can do the rest of the initialization
- `call`, where we do the forward computation

In [11]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[
            int(input_shape[-1]),
            self.num_outputs
        ])
        
    def call(self, _input):
        return tf.matmul(_input, self.kernel)

In [12]:
layer = MyDenseLayer(10)

In [14]:
_ = layer(tf.zeros([10, 5]))

print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


## Models: Composing layers

In [15]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding="same")
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1, 2, 3])

In [16]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [17]:
block.layers

In [18]:
len(block.variables)

18

In [19]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

In [20]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [21]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________